# Non-Linear Programming (NLP)

## What is Non-Linear Programming?
**Non-Linear Programming (NLP)** is an optimization problem in which:
- the **objective function is non-linear**
- **one or more constraints are non-linear**.

Non-linear functions may include polynomial, exponential, logarithmic, or trigonometric terms, making the optimization problem more complex than linear or quadratic programming.

---

## Challenges in Non-Linear Programming
NLP problems introduce several challenges, including:
- The presence of **local optima and a global optimum**, where solvers may converge to a locally optimal solution instead of the best overall solution (if you are familiar with training machine learning or deep learning models, this behavior may feel familiar)
- **Non-convexity**, unlike convex QP problems where a unique global optimum is guaranteed
- **Increased computational complexity** due to non-linear objective functions or constraints

--- 

## Production Cost Optimization Problem

Lets head back to the production cost problem from the [previous notebook](04_Quadratic_Programming.ipynb) and adjust it so it simulates NLP.

**Things that would be modified:**
* **The Objective Fucntion:**
    * Now lets say there would be an additional cost due to the machines overheat which makes electricity bill for cooling increases in a rate of $3e^{0.1x_1}$ and $5e^{0.15x_2}$.
* **The Constraints:**
    * Effective production is reduced at rates of $ (1 - \frac{1}{1 + x_1}) $ and $ (1 - \frac{1}{1 + x_2}) $, introducing non-linearity into the production constraint.

### Problem Description
A manufacturer has developed an empirical model of its production system and has determined that the daily production cost is given by:

$\text{cost} = 0.4x_1^2 - 5x_1 + x_2^2 - 6x_2 + 3e^{0.1x_1} + 5e^{0.15x_2} + 50$

where $x_1$ and $x_2$ are the decision variables representing the number of hours per day that Unit 1 and Unit 2 are operated, respectively. The objective is to determine the optimal operating hours for both units that minimize the total production cost.

The manufacturer must produce at least 20,000 units per day.  
Unit 1 is an older but durable machine that produces 2,700 units per hour and can operate for up to 18 hours per day.  
Unit 2 is a newer and faster machine that produces 3,600 units per hour but can operate for at most 9 hours per day.

Assuming the machine runs nonstop we would add production effeciency loss of $ (1 - \frac{1}{1 + x}) $ due to the machine overheating  

Note:  
Due to the non-linear cost function and the non-linear production efficiency, this problem is formulated as a **Non-Linear Programming (NLP)** problem.

### Mathematical Formulation
#### Objective Function
This is the objective function that we want to minimize  
$\text{cost} = 0.4x_1^2 - 5x_1 + x_2^2 - 6x_2 + 3e^{0.1x_1} + 5e^{0.15x_2} + 50$

#### Constraints
**1. Production target**  
$ 2700 \times x_1 \times(1 - \frac{1}{1 + x_1}) + 3600 \times x_2 \times (1 - \frac{1}{1 + x_2}) \geq 20000 $

The terms $ (1 - \frac{1}{1 + x_1}) $ and $ (1 - \frac{1}{1 + x_2}) $ represent production efficiency loss due to machine overheating. As operating hours increase, the effective production rate of each machine decreases exponentially. These efficiency factors scale the nominal production rates, resulting in a reduced number of units produced per day.

**2. Machine Runtime Limit**  
$ x_1 \leq 18 $  
$ x_2 \leq 9 $

**3. Lower Bounds (non negativity)**  
$ x_1 \geq 0 $  
$ x_2 \geq 0 $

## Pyomo Implementation (NLP)

### Import Libraries

In [1]:
import pyomo.environ as pyomo
from pyomo.environ import value

### Create the Model

In [2]:
model = pyomo.ConcreteModel()

### Create Variables

In [3]:
# Decision Variables
## x1 and x2 would be in hour units therefore making the domain non negative reals so that we can have values like 1.1 hours
model.x1 = pyomo.Var(domain=pyomo.NonNegativeReals, initialize=6)
model.x2 = pyomo.Var(domain=pyomo.NonNegativeReals, initialize=4)

### Objective Function

In [4]:
def obj_func(model):
    return 0.4 * model.x1**2 - 5 * model.x1 + model.x2**2 - 6*model.x2 + 3*pyomo.exp(0.001 * model.x1) + 5*pyomo.exp(0.0015 * model.x2) + 50

model.objective = pyomo.Objective(rule = obj_func,
                                  sense = pyomo.minimize
                                  )

### Constraints

In [5]:
model.c = pyomo.ConstraintList()

# produce at most 20000 units
model.c.add(expr= 2700 * model.x1 * (1 - (1/(1 + model.x2)))+ 3600 * model.x2 * (1 - (1/(1 + model.x2)))>= 20000)
# model.c.add(expr= 2700 * model.x1 + 3600 * model.x2 >= 20000)

# machine runtime limit
model.c.add(expr= model.x1 <= 18)
model.c.add(expr= model.x2 <= 9)

### Solver
We will use **IPOPT** because IPOPT is a general-purpose
**Nonlinear Programming (NLP)** solver.

### Solver
We use **IPOPT**, a general-purpose **Nonlinear Programming (NLP)** solver, to solve this optimization problem.  
IPOPT is well suited for problems with **nonlinear objective functions and constraints**, making it appropriate for this NLP formulation.

In [6]:
ipopt_solver = pyomo.SolverFactory("ipopt")

In [7]:
ipopt_results = ipopt_solver.solve(model)

In [8]:
print(ipopt_results)


Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.14.19\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.33562302589416504
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [9]:
print("ipopt solver results")
print(f"x1 = \t \t \t \t \t{value(model.x1)}")
print(f"x2 = \t \t \t \t \t{value(model.x2)}")
print(f"minimized cost = \t \t \t{value(model.objective)}")
print(f"Units produced =\t \t \t{2700 * value(model.x1) + 3600 * value(model.x2)}")
print(f"cost per unit produced (cost/unit) = \t{value(model.objective)/(2700 * value(model.x1) + 3600 * value(model.x2))}")

ipopt solver results
x1 = 	 	 	 	 	6.246226514006858
x2 = 	 	 	 	 	2.9962331176203567
minimized cost = 	 	 	33.41633953174843
Units produced =	 	 	27651.2508112518
cost per unit produced (cost/unit) = 	0.001208492872884821


### Results Intepretation
The NLP model suggests operating Unit 1 for approximately 6.25 hours and Unit 2 for 3.00 hours per day.
This results in a total production of around 27,651 units, exceeding the minimum requirement of 20,000 units.

The solver intentionally avoids higher utilization levels due to nonlinear overheating penalties and production slowdowns, which increase costs exponentially.
Compared to the QP formulation, the NLP solution reflects more realistic machine behavior by discouraging excessive operation and favoring moderate production levels.